# Clasificación de municipios (Similares)

Un municipio puede ser “similar” a otro según varias dimensiones, como puede ser su Índice global (aunque esta es demasiado general). En este caso se emplean los vectores de categorías (salud, educación, bienestar, economía…). Municipios similares serán los que tengan perfiles parecidos en estas dimensiones.

Los motivos son los siguientes:

- Están normalizadas (0–1).

- Resumen información real del municipio.

- Son robustas a ruido.

- Son interpretables (cuando veas un cluster sabrás por qué están juntos).

Así que el clustering se hará sobre estas 6 columnas.

In [1]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances

In [3]:
# Cargar el conjunto de datos
df = pd.read_csv("../website_data/df_indices_categorias.csv")

feature_cols = ["sanidad", "educacion", "transporte_conectividad", "economia_empleo", "bienestar", "ocio_cultura", 'servicios']
X = df[feature_cols].copy()
municipios = df["municipio"]

print(X.head())

    sanidad  educacion  transporte_conectividad  economia_empleo  bienestar  \
0  0.928053   0.999672                 0.936331         0.858728   0.987277   
1  0.492601   0.145777                 0.173221         0.828036   0.066332   
2  0.095430   0.420926                 0.393818         0.530654   0.347530   
3  0.567489   0.177480                 0.186984         0.552960   0.477802   
4  0.206436   0.156520                 0.160438         0.402615   0.492952   

   ocio_cultura  servicios  
0      0.292520   0.433487  
1      0.685570   0.798210  
2      0.213418   0.972913  
3      0.065463   0.699308  
4      0.901106   0.691688  


## PCA

- No dependes de clusters.
- Cada municipio tendrá su propio conjunto único de similares,

In [4]:
# PCA (2 O 3 componentes funcionanmuy bien)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Matriz de distancias
dist_matrix = cosine_distances(X_pca)

# Función para obtener los municipios más similares
k = 10  # Número de municipios similares a encontrar

similares = {}

for i, muni in enumerate(municipios):
    # ordenar por distancia (de menor a mayor)
    idx = dist_matrix[i].argsort()[1:k+1]
    similares[muni] = municipios.iloc[idx].tolist()

# Convertimos a tabla
df_similares = (
    pd.DataFrame.from_dict(similares, orient="index")
      .reset_index()
)

df_similares.columns = ["municipio", "sim1", "sim2", "sim3", "sim4", "sim5", "sim6", "sim7", "sim8", "sim9", "sim10"]

df_similares.to_csv("../website_data/municipios_similares.csv", index=False)
df_similares.head()

,municipio,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,sim10
0,La Acebeda,Madarcos,Horcajuelo de la Sierra,La Serna del Monte,Piñuécar-Gandullas,Braojos,Prádena del Rincón,Robregordo,Pinilla del Valle,Villavieja del Lozoya,Garganta de los Montes
1,Ajalvir,Meco,San Fernando de Henares,Galapagar,Mejorada del Campo,Velilla de San Antonio,Ciempozuelos,Torrejón de la Calzada,Torrelodones,Loeches,Paracuellos de Jarama
2,Alameda del Valle,Carabaña,Nuevo Baztán,Estremera,Rascafría,Corpa,Navalafuente,Cenicientos,Fuentidueña de Tajo,Valdelaguna,Anchuelo
3,El Álamo,Villar del Olmo,Santorcaz,Pezuela de las Torres,Titulcia,Cabanillas de la Sierra,Navalcarnero,Cadalso de los Vidrios,Valdetorres de Jarama,Buitrago del Lozoya,Molinos (Los)
4,Aldea del Fresno,Santos de la Humosa (Los),Chapinería,Santa María de la Alameda,San Martín de Valdeiglesias,Robledo de Chavela,Fresnedillas de la Oliva,Zarzalejo,Colmenar del Arroyo,Fresno de Torote,Morata de Tajuña


¿Por qué usar distancia coseno?

Porque es más estable cuando los datos están estandarizados.

Evita que un municipio con valores más grandes parezca “más lejano”

Captura muy bien el “perfil” de un municipio